In [1]:
import polars as pl
ruta = '/content/competencia_01_lags1y2_minmaxmean_0710.csv.gz'

df = pl.read_csv(
    ruta,
    infer_schema_length=10000,
    ignore_errors=True
)

df.shape

(978439, 2119)

In [2]:
resultado = (df.pivot(
        values="mpayroll",
        index="numero_de_cliente",
        on="foto_mes")
    .with_columns(
        ((pl.col("202106") - pl.col("202101"))/pl.col("202101")).alias("diff_mpayroll"))
    .select(["numero_de_cliente","diff_mpayroll"])
)
print(resultado)


shape: (168_881, 2)
┌───────────────────┬───────────────┐
│ numero_de_cliente ┆ diff_mpayroll │
│ ---               ┆ ---           │
│ i64               ┆ f64           │
╞═══════════════════╪═══════════════╡
│ 249221323         ┆ 0.763775      │
│ 249227600         ┆ 0.095417      │
│ 249234235         ┆ NaN           │
│ 249244449         ┆ NaN           │
│ 249244739         ┆ NaN           │
│ …                 ┆ …             │
│ 1598368433        ┆ null          │
│ 1598419415        ┆ null          │
│ 1598425905        ┆ null          │
│ 1598470389        ┆ null          │
│ 1598550167        ┆ null          │
└───────────────────┴───────────────┘


In [3]:
clientes_aguinaldo = (resultado.filter( 
        pl.col("diff_mpayroll").is_finite()
            & (pl.col("diff_mpayroll") >= 0.3)
            & (pl.col("diff_mpayroll") <= 1)
    ).select("numero_de_cliente")
)

df = df.with_columns(pl.when(pl.col("numero_de_cliente").is_in(clientes_aguinaldo["numero_de_cliente"].implode())
    )
    .then(1)
    .otherwise(0)
    .alias("cobra_aguinaldo")
)

In [4]:
df_aguinaldo = (df.filter(pl.col("cobra_aguinaldo") == 1)
      .pivot(values="mpayroll", index="numero_de_cliente", on="foto_mes")
)

df_aguinaldo = df_aguinaldo.with_columns(
    ((pl.col("202106") / pl.col("202101"))**(1/5) - 1).alias("tasa_geom")
)

meses = [202101, 202102, 202103, 202104, 202105, 202106]

for i, mes in enumerate(meses):
    df_aguinaldo = df_aguinaldo.with_columns(
        (pl.col("202101") * (1 + pl.col("tasa_geom"))**i).alias(f"payroll_sim_{mes}")
    )

In [5]:
df_aguinaldo

numero_de_cliente,202101,202102,202103,202104,202105,202106,tasa_geom,payroll_sim_202101,payroll_sim_202102,payroll_sim_202103,payroll_sim_202104,payroll_sim_202105,payroll_sim_202106
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
249221323,97524.39,97055.19,97289.79,97641.69,113852.55,172011.06,0.120182,97524.39,109245.075328,122374.377152,137081.585951,153556.337889,172011.06
249255456,249153.41,249153.41,308001.64,287404.94,272226.32,444353.51,0.122671,249153.41,279717.212315,314030.29509,352552.584871,395800.427674,444353.51
249255747,302546.99,335849.33,306581.01,298200.36,316818.63,534317.98,0.120473,302546.99,338995.729305,379835.557072,425595.480839,476868.239264,534317.98
249275806,330584.24,353408.47,295709.78,357705.17,292811.29,500672.76,0.086562,330584.24,359200.149806,390293.099335,424077.505176,460786.344166,500672.76
249321238,57837.11,63627.03,63627.03,63628.21,74296.64,111636.75,0.140565,57837.11,65966.99547,75239.659991,85815.738542,97878.445787,111636.75
…,…,…,…,…,…,…,…,…,…,…,…,…,…
1573977253,129528.44,125404.76,124798.46,124798.46,160105.45,221394.56,0.113167,129528.44,144186.797898,160503.999647,178667.771796,198887.08536,221394.56
1574076211,105966.47,105966.47,105966.47,108068.49,124279.35,187681.17,0.121116,105966.47,118800.676628,133189.307592,149320.628134,167405.705379,187681.17
1574833703,30543.74,31157.22,46409.74,46800.35,53737.47,52168.0,0.113003,30543.74,33995.278922,37836.852624,42112.536266,46871.385641,52168.0


In [6]:
df_long = df_aguinaldo.melt(id_vars=["numero_de_cliente", "tasa_geom"],  # columnas que se mantienen
    value_vars=[f"payroll_sim_{mes}" for mes in [202101, 202102, 202103, 202104, 202105, 202106]],  # columnas a “despivotar”
    variable_name="foto_mes",
    value_name="payroll_simulado"
)

df_long = df_long.with_columns(
    pl.col("foto_mes").str.replace("payroll_sim_", "").cast(pl.Int64)
)


/tmp/ipykernel_5055/1605197139.py:1: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_long = df_aguinaldo.melt(id_vars=["numero_de_cliente", "tasa_geom"],  # columnas que se mantienen


In [7]:
df_long

numero_de_cliente,tasa_geom,foto_mes,payroll_simulado
i64,f64,i64,f64
249221323,0.120182,202101,97524.39
249255456,0.122671,202101,249153.41
249255747,0.120473,202101,302546.99
249275806,0.086562,202101,330584.24
249321238,0.140565,202101,57837.11
…,…,…,…
1573977253,0.113167,202106,221394.56
1574076211,0.121116,202106,187681.17
1574833703,0.113003,202106,52168.0


In [8]:
df_final = df.join(df_long, on=["numero_de_cliente", "foto_mes"],how="left")
df_final.shape


(978439, 2122)

In [9]:
df_final.filter(pl.col('numero_de_cliente')==1574833703).select('mpayroll','tasa_geom','payroll_simulado')

mpayroll,tasa_geom,payroll_simulado
f64,f64,f64
30543.74,0.113003,30543.74
31157.22,0.113003,33995.278922
46409.74,0.113003,37836.852624
46800.35,0.113003,42112.536266
53737.47,0.113003,46871.385641
52168.0,0.113003,52168.0


In [10]:
df_final.head(5)

numero_de_cliente,foto_mes,clase_ternaria,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,…,Visa_msaldototal_delta1_mean,Visa_msaldototal_delta2_mean,Visa_msaldopesos_delta1_mean,Visa_msaldopesos_delta2_mean,Visa_msaldodolares_delta1_mean,Visa_msaldodolares_delta2_mean,Visa_mconsumospesos_delta1_mean,Visa_mconsumospesos_delta2_mean,Visa_mconsumosdolares_delta1_mean,Visa_mconsumosdolares_delta2_mean,Visa_mlimitecompra_delta1_mean,Visa_mlimitecompra_delta2_mean,Visa_madelantopesos_delta1_mean,Visa_madelantopesos_delta2_mean,Visa_madelantodolares_delta1_mean,Visa_madelantodolares_delta2_mean,Visa_fultimo_cierre_delta1_mean,Visa_fultimo_cierre_delta2_mean,Visa_mpagado_delta1_mean,Visa_mpagado_delta2_mean,Visa_mpagospesos_delta1_mean,Visa_mpagospesos_delta2_mean,Visa_mpagosdolares_delta1_mean,Visa_mpagosdolares_delta2_mean,Visa_fechaalta_delta1_mean,Visa_fechaalta_delta2_mean,Visa_mconsumototal_delta1_mean,Visa_mconsumototal_delta2_mean,Visa_cconsumos_delta1_mean,Visa_cconsumos_delta2_mean,Visa_cadelantosefectivo_delta1_mean,Visa_cadelantosefectivo_delta2_mean,Visa_mpagominimo_delta1_mean,Visa_mpagominimo_delta2_mean,cobra_aguinaldo,tasa_geom,payroll_simulado
i64,i64,str,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i32,f64,f64
249221323,202101,"""CONTINUA""",1,0,0,46,93,3070.24,15691.1,1278.11,890.96,636.86,8,1,1,0.0,-314.48,2,25761.53,0.0,10240.09,1,22890.77,1,5,35013.67,1,10,15929.92,1,13,27145.02,0,0.0,0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,0.120182,97524.39
249221323,202102,"""CONTINUA""",1,0,0,46,94,3385.85,19103.66,1230.47,1631.3,206.19,8,1,1,0.0,0.0,2,8116.43,0.0,10536.47,1,10629.59,1,1,2315.5,1,11,17285.39,1,22,25713.45,0,0.0,0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,0.120182,109245.075328
249221323,202103,"""CONTINUA""",1,0,0,46,95,5341.18,24020.21,755.85,2768.95,1140.11,8,1,1,0.0,-3933.18,2,4015.77,0.0,10789.84,1,4650.31,1,0,0.0,1,10,18976.09,1,16,45821.57,0,0.0,0,0.0,…,-22852.04,null,-26805.435,null,0.0,null,1979.81,null,0.085,null,0.0,null,0.0,null,0.0,null,-1,null,0.0,null,-14949.985,null,0.0,null,29,null,1979.81,null,0,null,0,null,-4779.975,null,1,0.120182,122374.377152
249221323,202104,"""CONTINUA""",1,0,0,46,96,5356.62,29189.81,1470.2,2627.7,685.53,8,1,1,0.0,-1960.57,2,2136.11,0.0,10974.0,1,9358.46,1,0,0.0,1,10,19201.77,1,18,17373.08,0,0.0,0,0.0,…,-2570.905,-25422.945,-3015.675,-29821.11,0.0,0.0,189.735,2169.545,-0.13,-0.045,0.0,0.0,0.0,0.0,0.0,0.0,2,1,-25366.72,-25366.72,25366.72,10416.735,0.0,0.0,30,60,189.735,2169.545,-1,0,0,0,-3706.68,-8486.655,1,0.120182,137081.585951
249221323,202105,"""""",1,0,0,46,97,2489.25,31505.78,1008.8,952.16,309.95,8,1,1,0.0,-263.5,2,6204.56,0.0,11106.55,1,13240.63,1,1,2216.97,1,9,19278.3,1,14,16708.69,0,0.0,0,0.0,…,9758.9,7187.995,11447.185,8431.51,0.0,0.0,-825.51,-635.775,-0.025,-0.155,0.0,0.0,0.0,0.0,0.0,0.0,2,4,0.0,-25366.72,16835.695,42202.415,0.0,0.0,30,61,-825.51,-635.775,0,-1,0,0,1055.7,-2650.98,1,0.120182,153556.33

In [11]:
df_final.write_csv("/content/dataset_1110_lag1y2_maxminmean_aguinaldo.csv.gz")
